In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_csv('AEP.csv')
df['Date'] = pd.to_datetime(df['Datetime'])
df['year'] = pd.DatetimeIndex(df['Date']).year
df['hour'] = pd.DatetimeIndex(df['Date']).hour

In [3]:
df_18 = df[df['year'] == 2018]
df_18['Days'] = range(len(df_18))

C:\Users\nicoc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [315]:
def mean(X, add = 0):
    return X + add

def squared_exponential(x, xi, l = 1, a = 1):
    rtn = np.exp(-(a*(x - xi))**2/(2*l**2))
    return rtn

def kernel_matrix(X, Xpr):
    j = np.array([squared_exponential(a, b) for a in Xpr['Days'] for b in X['Days']])
    return j.reshape((len(Xpr), len(X)))

def mean_predict(df, hr):
    #df = df[df['Days'] < day]
    correct_hour = df[df['hour'] == hr]
    return np.mean(correct_hour.AEP_MW)

In [316]:
start_index = np.random.uniform() * 365
start_index = int(np.floor(start_index))
X = df_18.iloc[start_index - 24: start_index + 24]
Xpr = df_18.iloc[start_index + 24:start_index + 48]

K_XX = kernel_matrix(X, X)
K_XXpr = kernel_matrix(X, Xpr)
K_XprX = kernel_matrix(Xpr, X)
K_XprXpr = kernel_matrix(Xpr, Xpr)
meanX = np.array([mean_predict(df_18, hr) for hr in X['hour']])
meanXpr = np.array([mean_predict(df_18, hr) for hr in Xpr['hour']])
y = X['AEP_MW']

In [317]:
mu = meanXpr + K_XXpr@np.linalg.inv(K_XX)@(y - meanX)
cov =  K_XprXpr - K_XXpr@np.linalg.inv(K_XX)@K_XprX

In [230]:
def mvnrand(μ, Σ, inflation=1e-6):
    x = np.random.multivariate_normal(μ, 100000*Σ + inflation*np.eye(Σ.shape[0]), 1)
    print(x)
    return x[0]

In [ ]:
plt.plot(range(24), mvnrand(mu, cov))
plt.plot(range(24), meanXpr)


[[13060.38700084 13041.70302768 12966.407455   13843.17302842
  14663.85595957 15417.7048027  15579.1284226  15378.03781748
  15677.47340185 15572.11183741 15569.88018789 15951.72524836
  16051.29447393 16041.08667112 16496.24248553 16652.01920179
  16181.45696787 16158.56810587 16466.93340242 16144.5002915
  15335.85013951 14624.69714167 14122.08788523 13792.59644717]]


In [173]:
display(K_XXpr)

array([[5.38018616e-032, 1.91547895e-029, 5.31109225e-027,
        1.14687658e-024, 1.92874985e-022, 2.52616378e-020,
        2.57675711e-018, 2.04697171e-016, 1.26641655e-014,
        6.10193668e-013, 2.28973485e-011, 6.69158609e-010,
        1.52299797e-008, 2.69957850e-007, 3.72665317e-006,
        4.00652974e-005, 3.35462628e-004, 2.18749112e-003,
        1.11089965e-002, 4.39369336e-002, 1.35335283e-001,
        3.24652467e-001, 6.06530660e-001, 8.82496903e-001],
       [1.17691094e-034, 5.38018616e-032, 1.91547895e-029,
        5.31109225e-027, 1.14687658e-024, 1.92874985e-022,
        2.52616378e-020, 2.57675711e-018, 2.04697171e-016,
        1.26641655e-014, 6.10193668e-013, 2.28973485e-011,
        6.69158609e-010, 1.52299797e-008, 2.69957850e-007,
        3.72665317e-006, 4.00652974e-005, 3.35462628e-004,
        2.18749112e-003, 1.11089965e-002, 4.39369336e-002,
        1.35335283e-001, 3.24652467e-001, 6.06530660e-001],
       [2.00500878e-037, 1.17691094e-034, 5.38018616e-